### Update database with fresh data from Binance

In [ ]:
# Every hour the script updates the database with the latest data from the API.
# The script is run using a cron job.

### Recalculate the additional columns